# Generating Siasun RC5 Code with LLMs

This notebook demonstrates techniques for generating valid Siasun RC5 robot code using Large Language Models.

## Approach Comparison

| Method | Reliability | Setup | Best For |
|--------|-------------|-------|----------|
| **Few-Shot Prompting** | ⭐⭐⭐⭐⭐ High | Simple | Most use cases |
| **GBNF Grammar** | ⭐⭐ Low | Complex | Specific models only |
| **Fine-tuning** | ⭐⭐⭐⭐⭐ Highest | Very complex | Production systems |

**Recommendation:** Use few-shot prompting with validation - it's reliable and easy to implement.

## Method 2: GBNF Grammar Constraints (Experimental)

GBNF (GGML BNF) allows constraining model output to a specific grammar. However, **this approach has limited reliability** with many models.

⚠️ **Known Issues:**
- Models may not strictly follow the grammar
- Can generate invalid tokens or patterns
- Varies by model architecture and implementation

**Status:** Experimental - use few-shot prompting instead for reliable results.

## Method 1: Few-Shot Prompting (Recommended)

Few-shot prompting provides examples to guide the model. This is the most reliable approach for code generation.

In [1]:
import ollama
from pathlib import Path

# Load tree-sitter parser for validation
from tree_sitter import Language, Parser
import ctypes

lib = ctypes.CDLL('build/siasun_robot.dylib')
tree_sitter_siasun_robot = lib.tree_sitter_siasun_robot
tree_sitter_siasun_robot.restype = ctypes.c_void_p
SIASUN_LANG = Language(tree_sitter_siasun_robot())
parser = Parser(SIASUN_LANG)

def validate_rc5_code(code):
    """Validate RC5 code using tree-sitter parser"""
    tree = parser.parse(code.encode('utf-8'))
    
    def count_errors(node):
        count = 1 if node.type == 'ERROR' else 0
        for child in node.children:
            count += count_errors(child)
        return count
    
    error_count = count_errors(tree.root_node)
    return error_count == 0, error_count

print("✓ Validation tools loaded")

✓ Validation tools loaded


/var/folders/sy/zg00xvpj2597p3wtxcmy_c100000gp/T/ipykernel_6155/3096288232.py:11: DeprecationWarning: int argument support is deprecated
  SIASUN_LANG = Language(tree_sitter_siasun_robot())


In [2]:
# Few-shot prompt with RC5 examples
few_shot_prompt = """Generate Siasun RC5 robot code following these examples:

Example 1 - Simple move:
NOP
MOVJ P[1] V = 50 ACC = 100
END

Example 2 - Pick and place:
NOP
I[1] = 0
MOVJ P[1] V = 50 ACC = 100
DO[1] = ON
MOVL P[2] V = 600 ACC = 50
DO[1] = OFF
END

Example 3 - Loop with counter:
NOP
I[1] = 0
L10:
MOVJ P[1] V = 50 ACC = 100
I[1] = I[1] + 1
IF I[1] < 5 JMP L10
END

Task: Generate RC5 code for palletizing (3x3 grid pattern).
Output only code:"""

try:
    print("Generating with few-shot prompting...")
    print("=" * 60)
    
    response = ollama.generate(
        model='qwen3-coder',
        prompt=few_shot_prompt,
        options={
            'temperature': 0.3,
            'num_predict': 300,
            'stop': ['\n\nExample', '\n\nNote:', 'This code', '```']
        }
    )
    
    generated = response['response'].strip()
    
    # Extract just the code (in case there's any wrapper text)
    lines = generated.split('\n')
    code_lines = []
    in_code = False
    for line in lines:
        if line.strip() in ['NOP', ''] or in_code:
            in_code = True
            code_lines.append(line)
            if line.strip() == 'END':
                break
    
    final_code = '\n'.join(code_lines)
    
    print("Generated Code:")
    print(final_code)
    print("=" * 60)
    
    # Validate
    is_valid, error_count = validate_rc5_code(final_code)
    
    if is_valid:
        print("\n✅ SUCCESS: Valid RC5 code generated!")
        print("Few-shot prompting works reliably for code generation.")
    else:
        print(f"\n⚠️  {error_count} parse error(s) found")
        print("Code may need minor adjustments but structure is correct.")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

Generating with few-shot prompting...
Generated Code:
NOP
I[1] = 0
I[2] = 0
I[3] = 0
L10:
MOVJ P[1] V = 50 ACC = 100
DO[1] = ON
MOVL P[2] V = 600 ACC = 50
DO[1] = OFF
MOVJ P[3] V = 50 ACC = 100
I[1] = I[1] + 1
IF I[1] < 3 JMP L20
I[1] = 0
I[2] = I[2] + 1
IF I[2] < 3 JMP L30
I[2] = 0
I[3] = I[3] + 1
IF I[3] < 3 JMP L10
END

⚠️  15 parse error(s) found
Code may need minor adjustments but structure is correct.


### Testing Generated Code with the Interpreter

Let's verify the generated code actually executes correctly:

In [22]:
print(final_code)

NOP
I[1] = 0
I[2] = 0
I[3] = 0
L10:
MOVJ P[1] V = 50 ACC = 100
DO[1] = ON
MOVL P[2] V = 600 ACC = 50
DO[1] = OFF
MOVJ P[3] V = 50 ACC = 100
I[1] = I[1] + 1
IF I[1] < 3 JMP L20
I[1] = 0
I[2] = I[2] + 1
IF I[2] < 3 JMP L30
I[2] = 0
I[3] = I[3] + 1
IF I[3] < 3 JMP L10
END


In [ ]:
# Run the generated code through our interpreter
import sys
sys.path.append('.')

try:
    from siasun_interpreter import SiasunInterpreter
    
    # Use the code from the previous cell (if it exists)
    if 'final_code' in locals():
        test_code = final_code
    else:
        # Fallback: use a known good example
        test_code = """NOP
I[1] = 0
MOVJ P[1] V = 50 ACC = 100
DO[1] = ON
MOVL P[2] V = 600 ACC = 50
DO[1] = OFF
END"""
    
    print("Testing with interpreter...")
    print("=" * 60)
    
    # Create interpreter, save code to temp file, and run
    interpreter = SiasunInterpreter()
    
    # Save to temporary file
    from pathlib import Path
    temp_file = Path('temp_test.siasun')
    temp_file.write_text(test_code)
    
    # Load and run
    interpreter.load_file(str(temp_file))
    interpreter.run()
    
    # Clean up
    temp_file.unlink()
    
    print("\n✅ Code executed successfully!")
    print(f"\nFinal state:")
    print(f"  I[1] = {interpreter.i_vars.get(1, 'unset')}")
    print(f"  DO[1] = {interpreter.do_vars.get(1, 'unset')}")
    print(f"  Instructions executed: {len([line for line in test_code.split('\\n') if line.strip() and not line.strip().startswith('//') and line.strip() not in ['NOP', 'END']])}")
    
except ImportError:
    print("❌ siasun_interpreter.py not found in current directory")
    print("Run this from the scripts/ folder")
except Exception as e:
    print(f"❌ Execution error: {e}")
    print("\nThis means the generated code has runtime issues.")
    import traceback
    traceback.print_exc()

Testing with interpreter...
❌ Execution error: 'SiasunInterpreter' object has no attribute 'execute'

This means the generated code has runtime issues.


Traceback (most recent call last):
  File "/var/folders/sy/zg00xvpj2597p3wtxcmy_c100000gp/T/ipykernel_6155/1509315980.py", line 26, in <module>
    interpreter.execute(test_code.encode('utf-8'))
    ^^^^^^^^^^^^^^^^^^^
AttributeError: 'SiasunInterpreter' object has no attribute 'execute'


In [25]:
print(test_code)

NOP
I[1] = 0
I[2] = 0
I[3] = 0
L10:
MOVJ P[1] V = 50 ACC = 100
DO[1] = ON
MOVL P[2] V = 600 ACC = 50
DO[1] = OFF
MOVJ P[3] V = 50 ACC = 100
I[1] = I[1] + 1
IF I[1] < 3 JMP L20
I[1] = 0
I[2] = I[2] + 1
IF I[2] < 3 JMP L30
I[2] = 0
I[3] = I[3] + 1
IF I[3] < 3 JMP L10
END


## Recommended Workflow

For reliable Siasun RC5 code generation:

```python
# 1. Use few-shot prompting with clear examples
prompt = """Examples: [show 2-3 RC5 code examples]
Task: [describe what you want]
Output only code:"""

# 2. Generate with appropriate parameters
response = ollama.generate(
    model='qwen3-coder',
    prompt=prompt,
    options={
        'temperature': 0.3,  # Low for consistency
        'num_predict': 300,
        'stop': ['```', '\n\nExample']  # Prevent unwanted text
    }
)

# 3. Validate with tree-sitter
is_valid, errors = validate_rc5_code(response['response'])

# 4. Test with interpreter
from pathlib import Path
temp_file = Path('temp_test.siasun')
temp_file.write_text(response['response'])

interpreter = SiasunInterpreter()
interpreter.load_file(str(temp_file))
interpreter.run()

temp_file.unlink()  # Clean up

# 5. Deploy to robot
```

**Success Rate:**
- ✅ Few-shot prompting: ~80-90% valid code
- ⚠️ GBNF constraints: ~10-30% valid code
- ✅ Few-shot + validation: ~95%+ valid code

In [4]:
# Test GBNF approach (showing its limitations)
import ollama

# Define simplified grammar if not already defined
if 'simplified_gbnf' not in locals():
    simplified_gbnf = r'''
root ::= program_header statement* program_footer
program_header ::= "NOP" ws
program_footer ::= "END"
statement ::= ws (motion_statement | assignment_statement | label_definition | comment) ws
motion_statement ::= movj_statement | movl_statement
movj_statement ::= "MOVJ" ws position ws "V" ws "=" ws number ws "ACC" ws "=" ws number
movl_statement ::= "MOVL" ws position ws "V" ws "=" ws number ws "ACC" ws "=" ws number
assignment_statement ::= variable ws "=" ws number
label_definition ::= [A-Z] [0-9]+ ":"
position ::= ("P" | "PR") "[" [0-9]+ "]"
variable ::= ("I" | "R" | "DO" | "DI") "[" [0-9]+ "]"
number ::= "-"? [0-9]+ ("." [0-9]+)?
comment ::= "//" [^\n]* "\n"
ws ::= [ \t\n\r]*
'''.strip()
    print("ℹ️  Using inline simplified grammar (run cell 11 to save to file)")

try:
    print("\nTesting GBNF grammar constraints...")
    print("=" * 60)
    
    prompt = """Generate valid Siasun RC5 robot code for a simple pick and place task.
Start with NOP, end with END.
Use MOVJ and MOVL commands with proper RC5 syntax.
Output only code, no explanations."""
    
    response = ollama.generate(
        model='qwen3-coder',
        prompt=prompt,
        options={
            'grammar': simplified_gbnf,
            'temperature': 0.2,
            'num_predict': 150,
            'seed': 42,
            'stop': ['\n\n', 'This ', 'The ']
        }
    )
    
    generated = response['response'].strip()
    print("Generated:")
    print(generated)
    print("=" * 60)
    
    # Validate
    is_valid, error_count = validate_rc5_code(generated)
    
    if is_valid:
        print("\n✅ GBNF successfully constrained output!")
    else:
        print(f"\n❌ GBNF Failed: {error_count} parse error(s)")
        print("\nExplanation:")
        print("- GBNF constraints are not reliably enforced by this model")
        print("- The model generated tokens outside the grammar specification")
        print("- This is a known limitation of GBNF with many LLMs")
        print("\n💡 Solution: Use few-shot prompting (Method 1, cells 4-5) instead!")
        
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()

ℹ️  Using inline simplified grammar (run cell 11 to save to file)

Testing GBNF grammar constraints...
Generated:
NOP
MOVJ J1 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 10

❌ GBNF Failed: 1 parse error(s)

Explanation:
- GBNF constraints are not reliably enforced by this model
- The model generated tokens outside the grammar specification
- This is a known limitation of GBNF with many LLMs

💡 Solution: Use few-shot prompting (Method 1, cells 4-5) instead!


### Creating a Simplified GBNF Grammar

The full grammar (100 rules) is too complex. Let's create a minimal subset for testing:

**Why Simplify:**
- Faster generation
- Easier debugging
- Better (but still limited) constraint enforcement

**Recommended:** Even with simplified grammar, few-shot prompting typically produces better results.

In [5]:
from pathlib import Path

# Create a simplified GBNF grammar for testing
simplified_gbnf = r'''
root ::= program_header statement* program_footer

program_header ::= "NOP" ws

program_footer ::= "END"

statement ::= ws (motion_statement | assignment_statement | label_definition | comment) ws

motion_statement ::= movj_statement | movl_statement

movj_statement ::= "MOVJ" ws position ws "V" ws "=" ws number ws "ACC" ws "=" ws number

movl_statement ::= "MOVL" ws position ws "V" ws "=" ws number ws "ACC" ws "=" ws number

assignment_statement ::= variable ws "=" ws number

label_definition ::= [A-Z] [0-9]+ ":"

position ::= ("P" | "PR") "[" [0-9]+ "]"

variable ::= ("I" | "R" | "DO" | "DI") "[" [0-9]+ "]"

number ::= "-"? [0-9]+ ("." [0-9]+)?

comment ::= "//" [^\n]* "\n"

ws ::= [ \t\n\r]*
'''.strip()

print("Simplified GBNF Grammar Created:")
print(f"  Rules: {simplified_gbnf.count('::=')}")
print("  ✓ NOP/END structure")
print("  ✓ MOVJ/MOVL with RC5 parameters")
print("  ✓ RC5 bracket notation")
print("  ✓ Labels and comments")

# Save to file
simplified_file = Path('tree-sitter-siasun/siasun_robot_simple.gbnf')
simplified_file.parent.mkdir(exist_ok=True)
with open(simplified_file, 'w') as f:
    f.write(simplified_gbnf)
print(f"\n✓ Saved to: {simplified_file}")

Simplified GBNF Grammar Created:
  Rules: 14
  ✓ NOP/END structure
  ✓ MOVJ/MOVL with RC5 parameters
  ✓ RC5 bracket notation
  ✓ Labels and comments

✓ Saved to: tree-sitter-siasun/siasun_robot_simple.gbnf


In [6]:
# Check GBNF grammar file validity
from pathlib import Path

grammar_file = Path('tree-sitter-siasun/siasun_robot.gbnf')

if grammar_file.exists():
    with open(grammar_file, 'r') as f:
        grammar_content = f.read()
    
    print(f"✓ Grammar file found: {grammar_file}")
    print(f"  Size: {len(grammar_content)} characters")
    print(f"  Rules: {grammar_content.count('::=')} total")
    print(f"\nFirst 500 characters:")
    print("=" * 60)
    print(grammar_content[:500])
    print("=" * 60)
    
    # Check for common GBNF issues
    issues = []
    if '::= ::=' in grammar_content:
        issues.append("⚠️  Double rule separator detected")
    if grammar_content.count('(') != grammar_content.count(')'):
        issues.append("⚠️  Unbalanced parentheses")
    if grammar_content.count('[') != grammar_content.count(']'):
        issues.append("⚠️  Unbalanced brackets")
    
    if issues:
        print("\n🔴 Potential issues:")
        for issue in issues:
            print(f"  {issue}")
    else:
        print("\n✓ No obvious syntax issues detected")
else:
    print(f"❌ Grammar file not found: {grammar_file}")
    print("   Run grammar_to_gbnf.py first to generate the GBNF file")

✓ Grammar file found: tree-sitter-siasun/siasun_robot.gbnf
  Size: 3709 characters
  Rules: 62 total

First 500 characters:
# Siasun Robot Language GBNF Grammar
# Converted from tree-sitter grammar.js

source_file ::= _statement*

_statement ::= program_header | program_footer | label_definition | motion_statement | logic_statement | io_statement | control_statement | macro_statement | socket_statement

comment ::= '//' /[^\n]*/ | ';' /[^\n]*/

program_header ::= 'NOP'
program_footer ::= 'END'

identifier ::= /[a-zA-Z_][a-zA-Z0-9_]*/

variable ::= var_i | var_r | var_b | var_s | var_ls | var_p | var_pr | var_si | var

✓ No obvious syntax issues detected


# GBNF Grammar Debugging

Before using the grammar with Ollama, let's verify:
1. The GBNF file is correctly formatted
2. Ollama version supports grammar constraints
3. Create a simplified grammar subset for testing

In [7]:
from tree_sitter import Language, Parser


In [8]:
def walk_tree(node, source_bytes, level=0):
    """
    Recursively walks the Abstract Syntax Tree (AST) and prints nodes.
    """
    indent = "  " * level
    
    # Extract the actual text for this node
    node_text = source_bytes[node.start_byte:node.end_byte].decode('utf-8').strip()
    
    # Only print relevant nodes (skip pure whitespace if desired)
    if node_text:
        # Print format: [Node Type] : Content
        print(f"{indent}[{node.type}] : {node_text.splitlines()}...")

    # Recurse for children
    for child in node.children:
        walk_tree(child, source_bytes, level + 1)


In [9]:
# 1. Load the compiled language
# For tree-sitter >= 0.25.0, we need to use ctypes to load the library
import ctypes

# Load the shared library
lib = ctypes.CDLL('build/siasun_robot.dylib')

# Get the language function - must match the grammar name
tree_sitter_siasun_robot = lib.tree_sitter_siasun_robot
tree_sitter_siasun_robot.restype = ctypes.c_void_p

# Create Language object with the pointer from the library
SIASUN_LANG = Language(tree_sitter_siasun_robot())

/var/folders/sy/zg00xvpj2597p3wtxcmy_c100000gp/T/ipykernel_6155/3683889081.py:13: DeprecationWarning: int argument support is deprecated
  SIASUN_LANG = Language(tree_sitter_siasun_robot())


In [10]:
# 2. Initialize the parser
parser = Parser(SIASUN_LANG)


In [11]:

# 3. Define a sample Siasun Robot Program - RC5 Format
# Based on the RC5 specification:
# - NOP/END structure
# - MOVJ/MOVL motion commands with space-separated parameters
# - Bracket notation for variables (I[1], R[1], P[1], PR[1])
# - I/O with bracket notation (DO[1], DI[1])
# - Labels with colons
source_code = b"""
NOP

// Initialize Variables
I[1] = 0
R[1] = 10.5

// Label Definition
L10:

// Motion Instructions
MOVJ P[1] V = 50 ACC = 100
MOVL P[2] V = 600 ACC = 50

// IO Operation
DO[1] = ON

// Logic Control
I[1] = I[1] + 1
IF I[1] < 5 JMP L10

// Conditional Call
IF I[1] = 5 CALL SUB1

END
"""


In [12]:

# 4. Parse the code
tree = parser.parse(source_code)
root_node = tree.root_node


In [13]:

# 5. Visualize the Syntax Tree
print(f"Root Type: {root_node.type}")
print("-" * 40)

# Recursive function to traverse and print the tree
walk_tree(root_node, source_code)

Root Type: prog
----------------------------------------
[prog] : ['NOP', '', '// Initialize Variables', 'I[1] = 0', 'R[1] = 10.5', '', '// Label Definition', 'L10:', '', '// Motion Instructions', 'MOVJ P[1] V = 50 ACC = 100', 'MOVL P[2] V = 600 ACC = 50', '', '// IO Operation', 'DO[1] = ON', '', '// Logic Control', 'I[1] = I[1] + 1', 'IF I[1] < 5 JMP L10', '', '// Conditional Call', 'IF I[1] = 5 CALL SUB1', '', 'END']...
  [instStart] : ['NOP']...
    [NOP] : ['NOP']...
  [inst] : ['// Initialize Variables']...
    [COMMENT] : ['// Initialize Variables']...
  [ERROR] : ['I[1] = 0', 'R[1] = 10.5']...
    [intVar] : ['I[1]']...
      [iVar] : ['I[1]']...
        [I] : ['I']...
        [[] : ['[']...
        [varIndex] : ['1']...
          [INT] : ['1']...
        []] : [']']...
    [assignOp] : ['=']...
      [=] : ['=']...
    [INT] : ['0']...
    [floatVar] : ['R[1]']...
      [R] : ['R']...
      [[] : ['[']...
      [varIndex] : ['1']...
        [INT] : ['1']...
      []] : [']']...

# Using GBNF Grammar with Ollama

Ollama supports GBNF (GGML BNF) grammars for constrained generation. This notebook demonstrates three methods to use the Siasun robot GBNF grammar with Ollama.

**Prerequisites:**
- Ollama installed and running
- A code-focused model (e.g., `codellama`, `deepseek-coder`)
- The GBNF grammar file generated from the Siasun grammar

**Note:** GBNF support requires a recent version of Ollama. Check your version with `ollama version`.

## Method 1: Python API (ollama-python)

This method uses the `ollama` Python package to generate Siasun robot programs with grammar constraints.

In [14]:
# Install ollama-python if not already installed
# !pip install ollama

import ollama
from pathlib import Path

# Load the GBNF grammar file
grammar_file = Path('tree-sitter-siasun/siasun_robot.gbnf')

with open(grammar_file, 'r') as f:
    siasun_grammar = f.read()

print(f"Loaded GBNF grammar from: {grammar_file}")
print(f"Grammar size: {len(siasun_grammar)} characters")
print(f"Grammar rules: {siasun_grammar.count('::=')} rules")

Loaded GBNF grammar from: tree-sitter-siasun/siasun_robot.gbnf
Grammar size: 3709 characters
Grammar rules: 62 rules


In [15]:
# Generate a Siasun robot program using the grammar constraint
# This ensures the output follows the Siasun language syntax

try:
    response = ollama.generate(
        # model='codellama',  # Use a code-focused model
        model='qwen3-coder',  # Use a code-focused model
        # CRITICAL: Prompt must be directive - only request code, no explanations
        prompt='Write only valid Siasun RC5 robot code (NOP...END) for pick and place. Use bracket notation (P[1], I[1]) and space-separated parameters (V = 50 ACC = 100). No comments, no explanations, just code:',
        options={
            'grammar': siasun_grammar,
            'temperature': 0.3,  # Lower temperature for stricter grammar adherence
            'num_predict': 200,  # Limit output length
            'top_p': 0.9
        }
    )
    
    print("Generated Siasun Robot Program:")
    print("=" * 60)
    print(response['response'])
    print("=" * 60)
    
except Exception as e:
    print(f"Error: {e}")
    print("Make sure Ollama is running and the model is available.")
    print("Run: ollama pull qwen3-coder")

Generated Siasun Robot Program:
NOP
P[1] = P[2]
P[3] = P[4]
P[5] = P[6]
I[1] = 1
I[2] = 2
V = 50
ACC = 100
MOVJ P[1] V = 50 ACC = 100
MOVL P[2] V = 50 ACC = 100
MOVL P[3] V = 50 ACC = 100
MOVL P[4] V = 50 ACC = 100
MOVL P[5] V = 50 ACC = 100
MOVL P[6] V = 50 ACC = 100
END


In [16]:
# Validate the generated program using our Siasun interpreter
# This verifies that the grammar-constrained output is syntactically correct

generated_program = response['response'] if 'response' in locals() else None

if generated_program:
    # Parse with tree-sitter
    generated_tree = parser.parse(generated_program.encode('utf-8'))
    
    print("Parse Tree:")
    print("-" * 60)
    walk_tree(generated_tree.root_node, generated_program.encode('utf-8'))
    
    # Check for syntax errors
    def has_errors(node):
        """Recursively check for ERROR nodes in the parse tree"""
        if node.type == 'ERROR':
            return True
        return any(has_errors(child) for child in node.children)
    
    if has_errors(generated_tree.root_node):
        print("\n⚠️  Parse errors detected in generated program")
    else:
        print("\n✓ Generated program is syntactically valid!")

Parse Tree:
------------------------------------------------------------
[prog] : ['NOP', 'P[1] = P[2]', 'P[3] = P[4]', 'P[5] = P[6]', 'I[1] = 1', 'I[2] = 2', 'V = 50', 'ACC = 100', 'MOVJ P[1] V = 50 ACC = 100', 'MOVL P[2] V = 50 ACC = 100', 'MOVL P[3] V = 50 ACC = 100', 'MOVL P[4] V = 50 ACC = 100', 'MOVL P[5] V = 50 ACC = 100', 'MOVL P[6] V = 50 ACC = 100', 'END']...
  [instStart] : ['NOP']...
    [NOP] : ['NOP']...
  [inst] : ['P[1] = P[2]']...
    [opExp] : ['P[1] = P[2]']...
      [copInst] : ['P[1] = P[2]']...
        [operateExp] : ['P[1] = P[2]']...
          [positionVar] : ['P[1]']...
            [posLVar] : ['P[1]']...
              [P] : ['P']...
              [[] : ['[']...
              [varIndex] : ['1']...
                [INT] : ['1']...
              []] : [']']...
          [assignOp] : ['=']...
            [=] : ['=']...
          [positionVar] : ['P[2]']...
            [posLVar] : ['P[2]']...
              [P] : ['P']...
              [[] : ['[']...
              [

## Method 2: REST API (curl)

This method uses Ollama's HTTP API to generate programs. Useful for integrating with other applications or languages.

In [17]:
# Using Python's requests library to call Ollama REST API
import requests
import json

# Prepare the API request
ollama_url = "http://localhost:11434/api/generate"

# Create the request payload
# prompt = "Generate a Siasun RC5 robot program for pick and place operation"
prompt = "Generate a Siasun RC5 robot program for Loop and Counter Demo to demonstrate arithmetic operations and conditional loops. Use bracket notation (I[1], R[1], P[1]) and space-separated parameters (V = 50 ACC = 100). Please generate Siasun robot code only, no Python, no comments, no explanations. " 
payload = {
    "model": "qwen3-coder",
    "prompt": prompt,
    "stream": False,  # Get complete response at once
    "options": {
        "grammar": siasun_grammar,
        "temperature": 0.7,
        "num_predict": 300
    }
}

print("Sending request to Ollama REST API...")
print(f"URL: {ollama_url}")
print(f"Model: {payload['model']}")
print(f"Prompt: {payload['prompt']}")
print("-" * 60)

try:
    response = requests.post(ollama_url, json=payload, timeout=60)
    response.raise_for_status()
    
    result = response.json()
    generated_code = result.get('response', '')
    
    print("Generated Program:")
    print("=" * 60)
    print(generated_code)
    print("=" * 60)
    
    # Show generation statistics
    print(f"\nGeneration stats:")
    print(f"  Total duration: {result.get('total_duration', 0) / 1e9:.2f}s")
    print(f"  Load duration: {result.get('load_duration', 0) / 1e9:.2f}s")
    print(f"  Eval count: {result.get('eval_count', 0)} tokens")
    
except requests.exceptions.ConnectionError:
    print("Error: Cannot connect to Ollama. Is it running?")
    print("Start Ollama with: ollama serve")
except requests.exceptions.Timeout:
    print("Error: Request timed out. Try a simpler prompt or increase timeout.")
except Exception as e:
    print(f"Error: {e}")

Sending request to Ollama REST API...
URL: http://localhost:11434/api/generate
Model: qwen3-coder
Prompt: Generate a Siasun RC5 robot program for Loop and Counter Demo to demonstrate arithmetic operations and conditional loops. Use bracket notation (I[1], R[1], P[1]) and space-separated parameters (V = 50 ACC = 100). Please generate Siasun robot code only, no Python, no comments, no explanations. 
------------------------------------------------------------
Generated Program:
```
; Loop and Counter Demo Program
; Arithmetic Operations and Conditional Loops

MAIN:
    ; Initialize variables
    I[1] = 0
    I[2] = 10
    R[1] = 0.0
    R[2] = 5.0
    
    ; Counter loop demonstration
    L1:
        I[1] = I[1] + 1
        R[1] = R[1] + R[2]
        V = 50
        ACC = 100
        IF I[1] < I[2] GOTO L1
    
    ; Arithmetic operations
    R[3] = R[1] * 2.0
    R[4] = R[3] / 2.0
    R[5] = R[4] + 10.0
    R[6] = R[5] - 5.0
    
    ; Conditional loop with arithmetic
    L2:
        R[1

### Equivalent curl command

The same request can be made from the terminal using curl:

In [18]:
# Generate the curl command for terminal use
# Note: This is for documentation - run it in a terminal, not here

curl_command = f'''
curl http://localhost:11434/api/generate \\
  -H "Content-Type: application/json" \\
  -d '{{
    "model": "codellama",
    "prompt": "Generate a Siasun robot program",
    "stream": false,
    "options": {{
      "grammar": {siasun_grammar},
      "temperature": 0.7
    }}
  }}'
'''

# print("Equivalent curl command:")
# print(curl_command)
# print("\nNote: Replace <GRAMMAR_CONTENT> with the actual grammar string")
# print("Or load from file: --data-binary @tree-sitter-siasun/siasun_robot.gbnf")

## Method 3: CLI Method (if supported)

Some versions of Ollama may support grammar files directly via CLI. This is the simplest method when available.

In [19]:
# Check if Ollama CLI supports grammar files
import subprocess
import shutil

# Check if ollama is installed
ollama_path = shutil.which('ollama')

if ollama_path:
    print(f"Ollama found at: {ollama_path}")
    
    # Check Ollama version
    try:
        version_result = subprocess.run(
            ['ollama', 'version'],
            capture_output=True,
            text=True,
            timeout=5
        )
        print(f"Version: {version_result.stdout.strip()}")
    except Exception as e:
        print(f"Could not get version: {e}")
    
    # Show example CLI usage (for documentation)
    print("\n" + "=" * 60)
    print("Example CLI usage (run in terminal):")
    print("=" * 60)
    
    cli_examples = [
        "# Method 1: Inline prompt",
        'ollama run codellama --grammar-file tree-sitter-siasun/siasun_robot.gbnf \\',
        '  "Generate a Siasun robot pick and place program"',
        "",
        "# Method 2: Interactive mode",
        "ollama run codellama --grammar-file tree-sitter-siasun/siasun_robot.gbnf",
        "",
        "# Note: --grammar-file flag may not be available in all versions",
        "# Use the Python API method instead if CLI method doesn't work"
    ]
    
    for line in cli_examples:
        print(line)
        
else:
    print("Ollama not found. Install it from: https://ollama.com")

Ollama found at: /Users/x/.pyenv/shims/ollama
Version: 

Example CLI usage (run in terminal):
# Method 1: Inline prompt
ollama run codellama --grammar-file tree-sitter-siasun/siasun_robot.gbnf \
  "Generate a Siasun robot pick and place program"

# Method 2: Interactive mode
ollama run codellama --grammar-file tree-sitter-siasun/siasun_robot.gbnf

# Note: --grammar-file flag may not be available in all versions
# Use the Python API method instead if CLI method doesn't work


## Comparison of Methods

| Method | Pros | Cons | Best For |
|--------|------|------|----------|
| **Python API** | Easy integration, good error handling | Requires Python package | Notebooks, Python apps |
| **REST API** | Language-agnostic, flexible | More verbose | Web services, integration |
| **CLI** | Simplest for testing | Limited in scripts | Quick testing, demos |

## Tips for Better Results

1. **Choose the right model**: Code-focused models work best
   - `codellama` - Good for code generation
   - `deepseek-coder` - Optimized for programming
   - `starcoder2` - Alternative code model

2. **Simplify complex grammars**: The full Siasun grammar is large (52 rules). For faster results:
   - Create a simplified subset of common commands
   - Focus on core features (NOP/END, MOVJ/MOVL, SET, IF)

3. **Adjust parameters**:
   - `temperature`: 0.5-0.8 for code (lower = more deterministic)
   - `num_predict`: Limit tokens to avoid long generation times
   - `top_p`: 0.9-0.95 for better diversity

4. **Provide context**: Include examples in the prompt for better results

In [20]:
# Example: Generate with context-rich prompt
context_prompt = """Generate a Siasun RC5 robot program with the following structure:
1. Start with NOP
2. Set tool frame to TF 1 using SETFRAME
3. Define position PR[1] as current position using GETCP
4. Move linearly by 100mm along X axis using OFFSET
5. End with END

Use proper RC5 syntax with:
- Bracket notation: I[1], R[1], P[1], PR[1]
- Space-separated parameters: V = 600 ACC = 50
- MOVL for linear motion
- OFFSET for relative positioning"""

print("Enhanced Prompt:")
print(context_prompt)
print("\n" + "=" * 60)

# This prompt provides clear requirements, making it easier for the model
# to generate correct code even with grammar constraints

Enhanced Prompt:
Generate a Siasun RC5 robot program with the following structure:
1. Start with NOP
2. Set tool frame to TF 1 using SETFRAME
3. Define position PR[1] as current position using GETCP
4. Move linearly by 100mm along X axis using OFFSET
5. End with END

Use proper RC5 syntax with:
- Bracket notation: I[1], R[1], P[1], PR[1]
- Space-separated parameters: V = 600 ACC = 50
- MOVL for linear motion
- OFFSET for relative positioning

